In [64]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [65]:
# Generate summary statistics
df_market_data = pd.read_csv(
    Path("~/Desktop/CryptoClustering/Resources/crypto_market_data.csv")
)

df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [66]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

---

### Prepare the Data

In [67]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
#Identify the numeric columns to normalize
columns_to_normalize = [
    "price_change_percentage_24h",
    "price_change_percentage_7d",
    "price_change_percentage_14d",
    "price_change_percentage_30d",
    "price_change_percentage_60d",
    "price_change_percentage_200d",
    "price_change_percentage_1y"
]
#Initialize the StandardScaler
scaler = StandardScaler()

#Fit and transform the selected columns
scaled_data = scaler.fit_transform(df_market_data[columns_to_normalize])

In [68]:
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(scaled_data, columns=columns_to_normalize, index=df_market_data.index)

# Copy the crypto names from the original DataFrame
crypto_names = df_market_data.index.tolist()

# Set the coin_id column as index
# (Already done earlier when we used index_col="coin_id" in read_csv)

# Display the scaled DataFrame
print(df_scaled)

    price_change_percentage_24h  price_change_percentage_7d  \
0                      0.508529                    0.493193   
1                      0.185446                    0.934445   
2                      0.021774                   -0.706337   
3                     -0.040764                   -0.810928   
4                      1.193036                    2.000959   
5                      0.891871                    1.327295   
6                      0.011397                    2.572251   
7                      0.102530                    1.508001   
8                      0.077497                    0.334297   
9                      0.448952                   -0.190684   
10                     0.331280                   -1.614844   
11                     0.034352                   -0.733026   
12                     0.155710                   -0.922491   
13                     0.262723                    1.792602   
14                     0.130050                   -0.04

---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [69]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

#Display the list to confirm
print(k_values)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [70]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=k, random_state=1)
# 2. Fit the model to the data using `df_market_data_scaled`
    model.fit(df_scaled)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(model.inertia_)

#Display the inertia list to confirm
print(inertia)


[287.0, 212.1233420748626, 165.1367523344681, 79.02243535120975, 66.41305075939437, 61.634088110682214, 46.074467441030215, 43.39269411734827, 32.913222659715245, 30.469111293270355, 26.48010137439893]


In [71]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k_values,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

#Display the DataFrame to confirm
print(df_elbow)


     k     inertia
0    1  287.000000
1    2  212.123342
2    3  165.136752
3    4   79.022435
4    5   66.413051
5    6   61.634088
6    7   46.074467
7    8   43.392694
8    9   32.913223
9   10   30.469111
10  11   26.480101


In [72]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
contrast_elbow = df_elbow.hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    width=800,
    height=400
)

contrast_elbow

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 
K= 4


---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [73]:
# Initialize the K-Means model using the best value for k
k_best = 4
model = KMeans(n_clusters=k_best, random_state=1)

In [74]:
# Fit the K-Means model using the scaled DataFrame
model.fit(df_scaled)

KMeans(n_clusters=4, random_state=1)

In [75]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
predictions = model.predict(df_scaled)

# Print the resulting array of cluster values.
print(predictions)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [76]:
# Create a copy of the scaled DataFrame
df_clusters = df_scaled.copy()

In [77]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_clusters['cluster'] = predictions

# Display the copy of the scaled DataFrame
print(df_clusters)

    price_change_percentage_24h  price_change_percentage_7d  \
0                      0.508529                    0.493193   
1                      0.185446                    0.934445   
2                      0.021774                   -0.706337   
3                     -0.040764                   -0.810928   
4                      1.193036                    2.000959   
5                      0.891871                    1.327295   
6                      0.011397                    2.572251   
7                      0.102530                    1.508001   
8                      0.077497                    0.334297   
9                      0.448952                   -0.190684   
10                     0.331280                   -1.614844   
11                     0.034352                   -0.733026   
12                     0.155710                   -0.922491   
13                     0.262723                    1.792602   
14                     0.130050                   -0.04

In [78]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
contrast_clusters = df_clusters.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster",
    hover_cols=["index"],
    title="Cryptocurrency Clusters: 24h vs 7d Price Change",
    width=800,
    height=400
)


---

### Optimize Clusters with Principal Component Analysis.

In [79]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [80]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
crypto_pca = pca.fit_transform(df_scaled)

# View the scaled PCA data
print(crypto_pca)

[[-0.60066733  0.84276006  0.46159457]
 [-0.45826071  0.45846566  0.95287678]
 [-0.43306981 -0.16812638 -0.64175193]
 [-0.47183495 -0.22266008 -0.47905316]
 [-1.15779997  2.04120919  1.85971527]
 [-0.51653377  1.38837748  0.80407131]
 [-0.45071134  0.51769912  2.84614316]
 [-0.34559977  0.72943939  1.47801284]
 [-0.64946792  0.43216514  0.60030286]
 [-0.75901394 -0.20119979 -0.21765292]
 [-0.24819846 -1.37625159 -1.46202571]
 [-0.43840762 -0.17533654 -0.6633884 ]
 [-0.69342533 -0.47381462 -0.52759693]
 [ 0.06049915  2.90940385  1.49857131]
 [-0.39335243 -0.10819197 -0.01275608]
 [-0.79617564 -0.49440875  1.08281169]
 [ 0.06407452 -1.26982514 -1.09882928]
 [-0.48901506 -0.73271912 -0.06254323]
 [-0.3062723   0.70341515  1.71422359]
 [-0.51352775 -0.14280239 -0.65656583]
 [-0.36212044 -0.98691441 -0.72875232]
 [-0.60426463  0.82739764  0.43931594]
 [-0.4132956  -0.67411527 -1.07662834]
 [-0.40748304 -0.21250655 -0.35142563]
 [ 0.60897382  0.56353212 -1.14874159]
 [-0.45021114 -0.15101945

In [81]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_

# View the explained variance
print(explained_variance)

[0.3719856  0.34700813 0.17603793]


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** approximately 0.895, or 89.5%.

In [82]:
# Create a new DataFrame with the PCA data.
df_crypto_pca = pd.DataFrame(
    data=crypto_pca,
    columns=["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original scaled DataFrame
df_crypto_pca.index = df_scaled.index

# Set the coin_id column as index


# Display the scaled PCA DataFrame
print(df_crypto_pca)

         PC1       PC2       PC3
0  -0.600667  0.842760  0.461595
1  -0.458261  0.458466  0.952877
2  -0.433070 -0.168126 -0.641752
3  -0.471835 -0.222660 -0.479053
4  -1.157800  2.041209  1.859715
5  -0.516534  1.388377  0.804071
6  -0.450711  0.517699  2.846143
7  -0.345600  0.729439  1.478013
8  -0.649468  0.432165  0.600303
9  -0.759014 -0.201200 -0.217653
10 -0.248198 -1.376252 -1.462026
11 -0.438408 -0.175337 -0.663388
12 -0.693425 -0.473815 -0.527597
13  0.060499  2.909404  1.498571
14 -0.393352 -0.108192 -0.012756
15 -0.796176 -0.494409  1.082812
16  0.064075 -1.269825 -1.098829
17 -0.489015 -0.732719 -0.062543
18 -0.306272  0.703415  1.714224
19 -0.513528 -0.142802 -0.656566
20 -0.362120 -0.986914 -0.728752
21 -0.604265  0.827398  0.439316
22 -0.413296 -0.674115 -1.076628
23 -0.407483 -0.212507 -0.351426
24  0.608974  0.563532 -1.148742
25 -0.450211 -0.151019 -0.647401
26 -0.764665 -0.517886  0.204990
27 -0.556315 -1.938209 -1.261776
28 -0.425147  0.492976  1.058048
29  2.6768

---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [83]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [84]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_scaled)
    inertia.append(model.inertia_)

#Display the inertia list to confirm
print(inertia)

[287.0, 198.5718180979813, 145.63574772192862, 79.02243535120975, 66.21778006000338, 59.13907002537009, 48.40631449041361, 38.99358798927969, 34.40059545400234, 28.57360256096785]


In [85]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

In [86]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA Data", xticks=k)
elbow_plot_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** k=4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No it does not differ

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [87]:
# Initialize the K-Means model using the best value for k
k = 4
model= KMeans(n_clusters=k, random_state=1)

In [88]:
# Fit the K-Means model using the PCA data
model.fit_predict(df_crypto_pca)

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [89]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
predicted_clusters = model.fit_predict(df_crypto_pca)

# Print the resulting array of cluster values.
print(predicted_clusters)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [90]:
# Create a copy of the scaled PCA DataFrame
df_clusters = df_crypto_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_clusters["cluster"] = predicted_clusters

# Display the copy of the scaled PCA DataFrame
print(df_clusters)

         PC1       PC2       PC3  cluster
0  -0.600667  0.842760  0.461595        2
1  -0.458261  0.458466  0.952877        2
2  -0.433070 -0.168126 -0.641752        0
3  -0.471835 -0.222660 -0.479053        0
4  -1.157800  2.041209  1.859715        2
5  -0.516534  1.388377  0.804071        2
6  -0.450711  0.517699  2.846143        2
7  -0.345600  0.729439  1.478013        2
8  -0.649468  0.432165  0.600303        2
9  -0.759014 -0.201200 -0.217653        0
10 -0.248198 -1.376252 -1.462026        0
11 -0.438408 -0.175337 -0.663388        0
12 -0.693425 -0.473815 -0.527597        0
13  0.060499  2.909404  1.498571        2
14 -0.393352 -0.108192 -0.012756        0
15 -0.796176 -0.494409  1.082812        2
16  0.064075 -1.269825 -1.098829        0
17 -0.489015 -0.732719 -0.062543        0
18 -0.306272  0.703415  1.714224        2
19 -0.513528 -0.142802 -0.656566        0
20 -0.362120 -0.986914 -0.728752        0
21 -0.604265  0.827398  0.439316        2
22 -0.413296 -0.674115 -1.076628  

In [91]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
contrast_clusters_pca = df_clusters.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="cluster",
    hover_cols=["index"],
    title="Cryptocurrency Clusters (PCA: PC1 vs PC2)",
    width=800,
    height=400
)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [92]:
# Composite plot to contrast the Elbow curves
# YOUR CODE HERE!
elbow_plot_pca + elbow_plot_pca

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [93]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!

# Combine both plots side by side
contrast_clusters + contrast_clusters_pca

:Layout
   .NdOverlay.I  :NdOverlay   [cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,index)
   .NdOverlay.II :NdOverlay   [cluster]
      :Scatter   [PC1]   (PC2,index)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 